In [ ]:
import sys; sys.path.append("..")
import IPython
if (ipy:= IPython.get_ipython()) is not None:
    ipy.run_line_magic("load_ext", "autoreload")
    ipy.run_line_magic("autoreload", "2")
import numpy as np
from scipy.optimize import minimize, Bounds
import time, os

import ElasticRods
import elastic_rods
from elastic_rods import PeriodicRod, RodMaterial, EnergyType
from linkage_vis import LinkageViewer as Viewer

from py_newton_optimizer import NewtonOptimizerOptions
import compute_vibrational_modes
from sparse_matrices import SuiteSparseMatrix, TripletMatrix

from tencers import *

from Tencers.viewers import HybridViewer
from Tencers.springs import *
from Tencers.state_saver import save_state, load_state
from Tencers.rods_IO import *
from Tencers.init import *

# In case knitro throws an error
# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Define target rod

In [ ]:
# Create target rod: helix

n_divisions = 100

t = np.linspace(0,3*np.pi,n_divisions,endpoint=True)

x = np.cos(t)
y = np.sin(t)
z = np.arange(len(x)) * 0.02

rod_points = np.column_stack([x, y, z]) * 10

rod = ElasticRod(rod_points)

rod_youngs_modulus = 133 # conversion to cm
material = RodMaterial('ellipse', rod_youngs_modulus, 0.5, [0.1, 0.1])  
rod.setMaterial(material)

rod.setRestKappas(np.zeros_like(rod.restKappas()))

print("Number of vertices: ", rod.numVertices())
print("Rod length: ", rod.characteristicLength(), "cm")
print("Rod thickness: ", rod.material(0).crossSectionHeight, "cm")
print("Rod Young modulus: ", rod.material(0).youngModulus * 100, "MPa")

# Create external frame
P = [-2,-2,-1]
Q = [-2,2,-1]
R = [2,2,-1]
S = [2,-2,-1]
T = [2,-2,3]
U = [2,2,3]
V = [-2,2,3]
W = [-2,-2,3]
out_rod_points = np.array([P,Q,R,S,T,U,V,W])*10
out_rod = ElasticRod(out_rod_points)
out_rod.setMaterial(material)

In [ ]:
open_rods = [out_rod,rod]
closed_rods = []
rods = [out_rod,rod]
target_rods = [out_rod,rod]

In [ ]:
# Visualize target rod and external frame
viewer1 = HybridViewer(target_rods, 400,300, wireframe=True)
viewer1.show()

# Cable initialization: remove compressed springs

In [ ]:
# Initialize springs
springs = []
attachment_vertices = []
init_stiffness = compute_initial_stiffness_er(rod)
print("Initial spring stiffness: ", init_stiffness)

# Dense network of springs
for i in range(rod.numVertices()):
    for j in range(i+2,rod.numVertices()):
        attachment_vertices.append(SpringAttachmentVertices(1,i,1,j))
        coordA = rod.deformedPoints()[i]
        coordB = rod.deformedPoints()[j]
        dist = np.linalg.norm(coordB-coordA)
        springs.append(Spring(coordA,coordB,init_stiffness,dist))
        if dist < 1e-6: # Make sure data is not corrupted and that no 2 points are at distance 0
            raise Exception("Error 2: intra-rod spring of length 0")

# Springs between rod and external frame            
for i in range(rod.numVertices()):
    for j in range(8):
        attachment_vertices.append(SpringAttachmentVertices(1,i,0,j))
        coordA = rod.deformedPoints()[i]
        coordB = out_rod.deformedPoints()[j]
        dist = np.linalg.norm(coordB-coordA)
        springs.append(Spring(coordA,coordB,init_stiffness,dist))

tencer = Tencer(open_rods,closed_rods,springs,attachment_vertices,target_rods)

In [ ]:
# Optimizer options (equilibrium)
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 1000
opt.hessianScaledBeta = False
opt.verbose = 1

# Fixed variables: rod endpoints and external frame
rod_offset = out_rod.numDoF()
last_var = rod_offset + rod.thetaOffset() - 3
fv = [rod_offset, rod_offset+1, rod_offset+2, last_var, last_var+1,last_var+2]
fixed_vars = [i for i in range(rod_offset)] + fv

In [ ]:
# Initialization: remove all compressed springs
t_init1 = time.time()
tencer = remove_compressed_springs(tencer,fixed_vars,opt)
t_init2 = time.time()
print("total time: ", t_init2 - t_init1)

In [ ]:
# Visualization
print("Number of springs: ",tencer.numRestVars())
viewer_init = HybridViewer([tencer],wireframe=True)
v_init = Viewer(rod, superView=viewer_init)
viewer_init.show()

In [ ]:
# Save state for potential future use
# save_state(tencer,"data/helix_init.pkl")

# Greedy decimation: remove springs with lowest force

In [ ]:
# Load state if needed
# tencer = load_state("data/helix_init.pkl")

In [ ]:
# Optimizer options (equilibrium)
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 1000
opt.hessianScaledBeta = False
opt.verbose = 0

# Fixed variables: rod endpoints and external frame
rod_offset = out_rod.numDoF()
last_var = rod_offset + rod.thetaOffset() - 3
fv = [rod_offset, rod_offset+1, rod_offset+2, last_var, last_var+1,last_var+2]
fixed_vars = [i for i in range(rod_offset)] + fv

In [ ]:
t_greedy1 = time.time()

# Set disance threshold
distance_threshold = 1e-7

# Here no alignment with the target rods is needed
def alignment_function(tencer,rods):
    return rods

# Greedy decimation step
tencer,aligned_rods = greedy_decimation_step(tencer,target_rods, alignment_function, opt, fixed_vars,distance_threshold=distance_threshold)

t_greedy_2 = time.time()

In [ ]:
# Visualization
viewer_greedy = HybridViewer([tencer],wireframe=True)
v_greedy = Viewer(rod, superView=viewer_greedy)
viewer_greedy.show()

In [ ]:
# Save state for potential future use
# save_state(tencer, "data/helix_greedy.pkl")

# Replace cables with smaller rest length springs

In [ ]:
# Load springs if needed
# tencer = load_state("data/helix_greedy.pkl")

In [ ]:
# Optimizer options (equilibrium)
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 1000
opt.hessianScaledBeta = False
opt.verbose = 1

# Fixed variables: rod endpoints and external frame
rod_offset = out_rod.numDoF()
last_var = rod_offset + rod.thetaOffset() - 3
fv = [rod_offset, rod_offset+1, rod_offset+2, last_var, last_var+1,last_var+2]
fixed_vars = [i for i in range(rod_offset)] + fv

In [ ]:
# Get current springs' rest lengths
rest_lengths = []
for spring in tencer.getSprings():
    rest_lengths.append(spring.get_rest_length())

In [ ]:
# Try reducing rest lengths while keeping the equilibrium stable
t_replace_springs1 = time.time()
for x in np.linspace(0,1,100):
    new_springs = replace_springs_with_rest_length(tencer.getSprings(),np.array(rest_lengths)*x)
    tencer1 = Tencer(tencer.getOpenRods(),tencer.getClosedRods(),new_springs,tencer.getAttachmentVertices(),tencer.getTargetRods())
    lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(tencer1, fixedVars=fixed_vars, mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)
    #print(lambdas[0])
    if lambdas[0]>-1e-10:
        x_opti = x
        print(x)
        break 
t_replace_springs2 = time.time()
print("Running time: ", t_replace_springs2 - t_replace_springs1)

In [ ]:
new_springs = replace_springs_with_rest_length(tencer.getSprings(),np.array(rest_lengths)*x_opti)
tencer = Tencer(tencer.getOpenRods(),tencer.getClosedRods(),new_springs,tencer.getAttachmentVertices(),tencer.getTargetRods())

In [ ]:
# Visualization
viewer_rep = HybridViewer([tencer],wireframe=True)
v_rep = Viewer(rod, superView=viewer_rep)
viewer_rep.show()

In [ ]:
# Save state for potential future use
# save_state(tencer, "data/helix_rep.pkl")

# Spring sparsification

In [ ]:
# Load state if needed
# tencer = load_state("data/helix_rep.pkl")

In [ ]:
# Optimizer options
opt_opts = NewtonOptimizerOptions()
opt_opts.useNegativeCurvatureDirection = True
opt_opts.niter = 10000
opt_opts.gradTol = 1e-6
opt_opts.verbose = 0
opt_opts.hessianScaledBeta = False

# Fixed variables: rod endpoints and external frame
rod_offset = out_rod.numDoF()
last_var = rod_offset + rod.thetaOffset() - 3
fv = [rod_offset, rod_offset+1, rod_offset+2, last_var, last_var+1,last_var+2]
fixed_vars = [i for i in range(rod_offset)] + fv

# This can be used to give a higher weight to some of the curve areas for target approximation
# Here we use a uniform weight
radii_array = [np.ones(out_rod.numVertices()),np.ones(rod.numVertices())]

# Optimization algorithm
algorithm = OptAlgorithm.NEWTON_CG

# Callback function: no need to realign here, do nothing
def newPt():
    return

In [ ]:
def num_inner_springs(knot:Tencer):
    v = knot.getAttachmentVertices()
    n = 0
    for x in v:
        if x.rodIdxA == x.rodIdxB:
            n = n+1
    return n

In [ ]:
# Sparsification weights - start with outer cables
# Start with 0: optimize shape before sparsifying
for outer_sparsification_weight in [0,1,10,1e2,1e3]:
    
    sparsification_weights = [0]*num_inner_springs(tencer) + [outer_sparsification_weight] * (tencer.numRestVars()-num_inner_springs(tencer))
    
    # Optimization object
    tencer_opt = TencerOptimization(tencer=tencer,
                                      Newton_optimizer_options=opt_opts,
                                      fixed_vars=fixed_vars,
                                      open_target_rod=target_rods,
                                      closed_target_rod=[],
                                      radii=radii_array,
                                      sparsification_weights = sparsification_weights,
                                      hessian_shift=1e-6)

    # Run optimization
    t_sparsification1 = time.time()
    optimize(tencer_opt,algorithm,100,0.005, 1e-5,newPt)
    t_sparsification2 = time.time()

    # Remove springs with zero stiffness
    tencer = remove_zero_springs(tencer)

    print("Running time: ", t_sparsification2 - t_sparsification1)
    print("Remaining springs: ", tencer.numRestVars())

In [ ]:
# Sparsification weights - continue with inner cables
# Start with 0: optimize shape before sparsifying
for inner_sparsification_weight in [0,1,10,1e2]:
# for inner_sparsification_weight in [0,1,10,1e2,1e3]: # this allows to go down to 7 springs, at the cost of being further from the target
    
    sparsification_weights = [inner_sparsification_weight]*num_inner_springs(tencer) + [0] * (tencer.numRestVars()-num_inner_springs(tencer))
    
   # Optimization object
    tencer_opt = TencerOptimization(tencer=tencer,
                                      Newton_optimizer_options=opt_opts,
                                      fixed_vars=fixed_vars,
                                      open_target_rod=target_rods,
                                      closed_target_rod=[],
                                      radii=radii_array,
                                      sparsification_weights = sparsification_weights,
                                      hessian_shift=1e-6)

    # Run optimization
    t_sparsification1 = time.time()
    optimize(tencer_opt,algorithm,100,0.005, 1e-5,newPt)
    t_sparsification2 = time.time()

    # Remove springs with zero stiffness
    tencer = remove_zero_springs(tencer)

    print("Running time: ", t_sparsification2 - t_sparsification1)
    print("Remaining springs: ", tencer.numRestVars())

In [ ]:
# Check no spring is compressed
v = tencer.getRestVars()
s = np.array(tencer.getSprings())
tensionned_springs = []
compressed_springs = []
for i,spring in enumerate(s):
    spring_length = np.linalg.norm(spring.get_coords()[:3] - spring.get_coords()[3:])
    spring_rest_length = spring.get_rest_length()
    if spring_length < spring_rest_length:
        compressed_springs.append(i)
    else: 
        tensionned_springs.append(i)
compressed_springs

In [ ]:
tencer = remove_compressed_springs(tencer,fixed_vars,opt)

In [ ]:
viewer_sp = HybridViewer([tencer],wireframe=True)
v_sp1 = Viewer(rod, superView=viewer_sp)
viewer_sp.show()

In [ ]:
c = computeEquilibrium(tencer,fixed_vars,opt,hessianShift = 1e-8)

In [ ]:
# Save state for potential future use
# save_state(tencer,"data/helix_sparse.pkl")

# Adding endpoints

In [ ]:
# Load state if needed
# tencer = load_state("data/helix_sparse.pkl")

In [ ]:
# Optimizer options (equilibrium)
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 1000
opt.hessianScaledBeta = False
opt.verbose = 1

# Fixed variables: external frame - without the rod endpoints
rod_offset = out_rod.numDoF()
fixed_vars = [i for i in range(rod_offset)]

In [ ]:
last_var = rod_offset + rod.thetaOffset() 
forces = -tencer.gradient(knotEnergyType = KnotEnergyType.Full, energyType = EnergyType.Full,vmask = VariableMask.Defo)
elastic_forces = forces[rod_offset:last_var].reshape(-1, 3)

In [ ]:
point_positions = tencer.getOpenRods()[1].deformedPoints()
springs = []
attachment_vertices = []

In [ ]:
def find_basis(force_vector, point_position, out_rod_points):
    
    vect = out_rod_points - point_position.reshape(1,3)
    
    comb = combinations(np.arange(len(out_rod_points)),3)
    
    for c in comb:
        X = np.column_stack(vect[np.array(c)])
        try:
            X_1 = np.linalg.inv(X)
        except:
            print("could not invert matrix for combination: ", c)
        res = X_1 @ force_vector
        if np.all(res > 0):
            return res, np.array(c)

In [ ]:
from itertools import combinations
for i in [0,n_divisions-1]:
    coords, basis = find_basis(-elastic_forces[i],point_positions[i],out_rod_points)
    print(basis)
    springs.append(Spring(rod_points[i],out_rod_points[basis[0]],coords[0],0))
    springs.append(Spring(rod_points[i],out_rod_points[basis[1]],coords[1],0))
    springs.append(Spring(rod_points[i],out_rod_points[basis[2]],coords[2],0))
    attachment_vertices.append(SpringAttachmentVertices(1,i,0,basis[0]))
    attachment_vertices.append(SpringAttachmentVertices(1,i,0,basis[1]))
    attachment_vertices.append(SpringAttachmentVertices(1,i,0,basis[2]))

In [ ]:
final_springs = tencer.getSprings() + springs
final_attachment_vertices = tencer.getAttachmentVertices() + attachment_vertices
tencer = Tencer(tencer.getOpenRods(),[],final_springs,final_attachment_vertices,target_rods)

In [ ]:
opt.verbose = 1
c = computeEquilibrium(tencer,fixedVars=fixed_vars, opts=opt, hessianShift = 1e-8)

In [ ]:
# Save state for potential future use
# save_state(tencer,"data/helix_endpoints.pkl")

# Optimize with sliding nodes

In [ ]:
# Load state if needed
# tencer = load_state("data/helix_endpoints.pkl")

In [ ]:
# Get current springs' rest lengths
rest_lengths = []
for spring in tencer.getSprings():
    rest_lengths.append(spring.get_rest_length())
    
# Try reducing rest lengths while keeping the equilibrium stable
t_replace_springs1 = time.time()
for x in np.linspace(0,1,100):
    new_springs = replace_springs_with_rest_length(tencer.getSprings(),np.array(rest_lengths)*x)
    tencer1 = Tencer(tencer.getOpenRods(),tencer.getClosedRods(),new_springs,tencer.getAttachmentVertices(),tencer.getTargetRods())
    lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(tencer1, fixedVars=fixed_vars, mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)
    #print(lambdas[0])
    if lambdas[0]>-1e-10:
        x_opti = x
        print(x)
        break 
t_replace_springs2 = time.time()
print("Running time: ", t_replace_springs2 - t_replace_springs1)

new_springs = replace_springs_with_rest_length(tencer.getSprings(),np.array(rest_lengths)*x_opti)
tencer = Tencer(tencer.getOpenRods(),tencer.getClosedRods(),new_springs,tencer.getAttachmentVertices(),tencer.getTargetRods())

In [ ]:
# Optimizer options
opt_opts = NewtonOptimizerOptions()
opt_opts.useNegativeCurvatureDirection = True
opt_opts.niter = 10000
opt_opts.gradTol = 1e-6
opt_opts.verbose = 0
opt_opts.hessianScaledBeta = False

# Fixed variables: rod endpoints and external frame
rod_offset = out_rod.numDoF()
last_var = rod_offset + rod.thetaOffset() - 3
fv = [rod_offset, rod_offset+1, rod_offset+2, last_var, last_var+1,last_var+2]
fixed_vars = [i for i in range(rod_offset)] + fv

# This can be used to give a higher weight to some of the curve areas for target approximation
# Here we use a uniform weight
radii_array = [np.ones(out_rod.numVertices()),np.ones(rod.numVertices())]

# Optimization algorithm
algorithm = OptAlgorithm.NEWTON_CG

# Callback function: no need to realign here, do nothing
def newPt():
    return

# Here we are optimizing over the springs' positions: the rest variables type must be set to StiffnessAndSpringAnchors
tencer.setRestVarsType(RestVarsType.StiffnessAndSpringAnchors)

In [ ]:
# Optimization object
tencer_opt = TencerOptimization(tencer=tencer,
                                  Newton_optimizer_options=opt_opts,
                                  fixed_vars=fixed_vars,
                                  open_target_rod=target_rods,
                                  closed_target_rod=[],
                                  radii=radii_array,
                                  hessian_shift=1e-6)



# Run optimization
t_sn1 = time.time()
optimize(tencer_opt,algorithm,100,0.005, 1e-5,newPt)
t_sn2 = time.time()


print("Running time: ", t_sn2 - t_sn1)

In [ ]:
viewer_sn = HybridViewer([tencer],wireframe=True)
v_sn1 = Viewer(rod, superView=viewer_sn)
viewer_sn.show()

In [ ]:
# Save result to obj
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)
export_knot_to_obj(f"{output_dir}/helix_knitro", tencer)